In [1]:
using CSV, DataFrames
using FileIO, Dates
using DelimitedFiles

include("../region_cube_split/cubes.jl")
include("../region_network_create/network.jl")

add_properties (generic function with 1 method)

In [2]:
romania_full = CSV.read("../data/romania.csv", DataFrame);
romania = romania_full[romania_full.Datetime .> DateTime(1976,1,1,0,0,0),:];

vrancea = romania[(romania.Latitude .>= 45.20) .& (romania.Latitude .<= 46.0) .& 
                    (romania.Longitude .>= 26.0) .& (romania.Longitude .<= 27.0) .&
                    (romania.Depth .>= 50.0) .& (romania.Depth .<= 200.0), :];

In [3]:
vrancea, vrancea_cubes = region_cube_split(vrancea,side=5,energyRelease=true);

In [4]:
MG = create_network(vrancea, vrancea_cubes; edgeWeight=true)
# MG = add_properties(MG, vrancea_cubes)

{1727, 7612} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [5]:
motif = "Triangle"
# motif = "Tetrahedron"

"Triangle"

In [6]:
filename= "./motifs" * motif * "_vrancea_5km.csv"

"./motifsTriangle_vrancea_5km.csv"

In [7]:
# motifs = CSV.read(filename, DataFrame;header=false)

# Creates a matrix: number of motifs x motif nodes (3 for triangles, 4 for tetrahedrons)
motifs = readdlm(filename, ',', Int64);
motifs;


In [8]:
# Calculate total energy and mean energy only for cubes that are part of motifs
# This is done to reduce calculations. I do not need to calculate these for each motif
# (because, nodes are repeated in motifs.)
# (I calculate separately, then I just add them up for each motif)

# Get all the cubes, uniquely
vec_motifs = unique!(vec(motifs));

# Will apend to a dictionary: cubeIndex => [totalEnergy, meanEnergy]
cube_energy = Dict()
for value in vec_motifs
    # Get the cube from df_cubes
    respective_cube = vrancea_cubes.cubeIndex[value]
    # Use the cube to get all the quakes in that cube from df
    quakesInNode = vrancea[vrancea.cubeIndex .== respective_cube,:];

    # Calculate total energy and mean energy
    totalEnergy = sum(quakesInNode.energyRelease)
    meanEnergy = totalEnergy / length(quakesInNode.energyRelease)

    # Push to dictionary
    cube_energy[value] = [totalEnergy, meanEnergy]
end

In [9]:
# Caclulate total energy and mean energy per each motif
motif_energy = Dict()
for i=1:length(motifs[:,1])
    # initialize with zero
    total_energy_in_motif=0
    mean_energy_in_motif=0
    # parse each motif (3 elements for Triangles, 4 for Tetrahedrons)
    for j=1:length(motifs[1,:])
        # add to total energy and mean energy
        total_energy_in_motif += cube_energy[motifs[i,j]][1] 
        mean_energy_in_motif += cube_energy[motifs[i,j]][2]  
    end
    # Add to dictionary
    motif_energy[i] = [total_energy_in_motif,mean_energy_in_motif]
end

In [10]:
motif_energy

Dict{Any, Any} with 2312 entries:
  1144 => [1.06164e12, 4.3573e10]
  2108 => [3.06265e12, 1.03199e11]
  1175 => [3.11695e12, 1.12903e11]
  1953 => [2.19417e12, 8.58461e10]
  719  => [4.99602e11, 4.36058e10]
  1546 => [1.3779e12, 6.83807e10]
  1703 => [6.48054e12, 1.74651e11]
  1956 => [2.49914e12, 1.37161e11]
  2261 => [3.07928e12, 9.16835e10]
  2288 => [6.98603e12, 1.94934e11]
  1028 => [9.37478e11, 5.65458e10]
  699  => [2.09455e12, 6.99485e10]
  831  => [1.03681e12, 3.69834e10]
  1299 => [1.29191e12, 8.8249e10]
  1438 => [1.2565e12, 8.09358e10]
  1074 => [1.58424e12, 5.54168e10]
  319  => [4.62493e12, 1.27889e11]
  687  => [1.14613e12, 1.0504e11]
  1812 => [3.50642e12, 1.46098e11]
  ⋮    => ⋮

In [13]:
using Geodesy

## AREAS

In [14]:
areas = Dict()
zeros=[]
for i=1:length(motifs[:,1])
    x = Vector{Any}(undef,3)
    for j=1:3

        lat = vrancea_cubes.cubeLatitude[motifs[i,j]]
        lon = vrancea_cubes.cubeLongitude[motifs[i,j]]
        dep = vrancea_cubes.cubeDepth[motifs[i,j]]

        x[j] = LLA(lat,lon,-1000*dep)

    end

    a = Geodesy.euclidean_distance(x[1],x[2]) / 1000
    b = Geodesy.euclidean_distance(x[2],x[3]) / 1000
    c = Geodesy.euclidean_distance(x[1],x[3]) / 1000;

    # calculate semiperimeter
    sp = (a+b+c)/2
            
    # Use Heron's formula Area = sqrt(semiperimeter(sp-a)(sp-b)(sp-c))
    # We have the areas of the triangle, append them to lists ! 

    A = sqrt(abs(sp*(sp-a)*(sp-b)*(sp-c)))
    if A == 0
        push!(zeros,i)
    end

    areas[i] = A
end

## VOLUMES

In [126]:
filename="./motifsTetrahedron_vrancea_5km.csv"

"./motifsTetrahedron_vrancea_5km.csv"

In [128]:
# motifs = CSV.read(filename, DataFrame;header=false)

# Creates a matrix: number of motifs x motif nodes (3 for triangles, 4 for tetrahedrons)
motifs = readdlm(filename, ',', Int64);
motifs;


### Volumes

In [130]:
volumes = Dict()
for i=1:length(motifs[:,1])
    x = Vector{Any}(undef,4)
    for j=1:4

        lat = vrancea_cubes.cubeLatitude[motifs[i,j]]
        lon = vrancea_cubes.cubeLongitude[motifs[i,j]]
        dep = vrancea_cubes.cubeDepth[motifs[i,j]]

        x[j] = LLA(lat,lon,-dep)

    end

    W = Geodesy.euclidean_distance(x[1],x[2]) / 1000
    V = Geodesy.euclidean_distance(x[2],x[3]) / 1000
    U = Geodesy.euclidean_distance(x[1],x[3]) / 1000
    u = Geodesy.euclidean_distance(x[2],x[4]) / 1000
    v = Geodesy.euclidean_distance(x[1],x[4]) / 1000
    w = Geodesy.euclidean_distance(x[3],x[4]) / 1000

    # calculate elements that go into elements that go into Heron formula
    A = (w-U+v)*(U+v+w)
    B = (u-V+w)*(V+w+u)
    C = (v-W+u)*(W+u+v)
    
    a = (U-v+w)*(v-w+U)
    b = (V-w+u)*(w-u+V)
    c = (W-u+v)*(u-v+W)

    # elements that go into Heron formula
    p = sqrt(abs(a*B*C))
    q = sqrt(abs(b*C*A))
    r = sqrt(abs(c*A*B))
    s = sqrt(abs(a*b*c))
            
    # Use Heron's formula Area = sqrt(semiperimeter(sp-a)(sp-b)(sp-c))
    # We have the areas of the triangle, append them to lists ! 

    V = sqrt(abs((-p+q+r+s)*(p-q+r+s)*(p+q-r+s)*(p+q+r-s)))/(192*u*v*w)

    volumes[i] = V
end

In [131]:
volumes

Dict{Any, Any} with 21977 entries:
  11950 => 1.023
  1703  => 1.07235
  12427 => 1.54003
  7685  => 0.225364
  18374 => 0.159408
  3406  => 0.247584
  1090  => 0.588097
  2015  => 0.54375
  18139 => 1.53502
  17088 => 0.174772
  11280 => 0.139509
  16805 => 1.13957
  3220  => 0.661264
  11251 => 0.614917
  422   => 0.206537
  15370 => 0.0909343
  15859 => 0.255473
  4030  => 0.231534
  8060  => 0.156393
  ⋮     => ⋮

In [136]:
# Define the function to do the regression with 
function power_law(x, a, b)
    return a*x^-b
end

power_law (generic function with 1 method)